In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tensorflow.python.framework import ops
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.model_selection import GridSearchCV

In [2]:
class SimpleSVM(BaseEstimator, ClassifierMixin): 
    """SVM Classifier implemented with Tensorflow and compatible to SkLearn Estimator"""
    
    def __init__(self, C=1.0, verbose=False) :
        self.C = C
        self.verbose = verbose

    def fit(self, X, y) : 
        X, y = check_X_y(X, y)
        
        # Step 1: Reinitialize the Tensorflow graph
        ops.reset_default_graph()
        self.sess_ = tf.Session()
        
        # Step 2: Create the Placeholders
        featuresSize = np.shape(X)[1]
        self.xInput_ = tf.placeholder(shape=[None, featuresSize], dtype=np.float32)
        yInput = tf.placeholder(shape=[None, 1], dtype=np.int32)
        
        # Step 3: Initialize the model parameters
        self.W_ = tf.Variable(tf.random_normal(shape=[featuresSize, 1], dtype=np.float32))
        self.b_ = tf.Variable(tf.random_normal(shape=[1, 1], dtype=np.float32))
        
        # Step 4: Compute the model output
        model_output = tf.add(tf.matmul(self.xInput_, self.W_), self.b_)
        
        # Step 5: Compute the Loss
        lossSecondPart = tf.reduce_sum(tf.maximum(0.0, tf.subtract(1.0, tf.multiply(model_output, tf.cast(yInput, dtype=np.float32)))))
        loss = tf.add(tf.multiply(0.5, tf.reduce_sum(tf.square(self.W_))), tf.multiply(self.C, lossSecondPart))
        
        # Step 6: Optimize our Loss
        optimizer = tf.train.GradientDescentOptimizer(0.01)
        train_step = optimizer.minimize(loss)
        
        # Step 7: Compute the model prediction
        self.prediction_ = tf.sign(tf.squeeze(model_output))
        
        # Step 8: Initialize the session
        init = tf.global_variables_initializer()
        self.sess_.run(init)
        
        # Step 9: Train the model
        epochs = 500
        batchSize = 500
        np.random.seed(seed=2)
        for epoch in range(epochs) :
            stepCounter = 0
            indexes = np.arange(len(y))
            np.random.shuffle(indexes)
            currIndex = batchSize
            while currIndex <= len(y) :
                stepCounter = stepCounter + 1
                currIndexes = indexes[currIndex-batchSize:currIndex]
                batchX = X[currIndexes]
                batchY = np.reshape(y[currIndexes], newshape=[-1, 1])
                
                _, lossVal, predictionVal = self.sess_.run([train_step, loss, self.prediction_], feed_dict={self.xInput_:batchX, yInput:batchY})
                f1Score = f1_score(batchY, predictionVal, average='weighted')
                accuracy = accuracy_score(batchY, predictionVal)
                if self.verbose :
                    print("Epoch {0}, Step {1}    Loss: {2}    F1_Score: {3}    Accuracy: {4}".format(epoch, stepCounter, lossVal, f1Score, accuracy))
                currIndex = currIndex + batchSize
        
        if self.verbose :
            print("End of the training")
            
        return self
        
    def predict(self, X) :
        X = check_array(X)
        check_is_fitted(self, ['W_'])
        output = self.sess_.run(self.prediction_, feed_dict={self.xInput_:X})  
        return output
    
    def score(self, X, y) :
        prediction = self.predict(X)
        accuracy = accuracy_score(y, prediction)
        if self.verbose :
            print("Accuracy Score is: {0}".format(accuracy))
        return f1_score(y, prediction, average='weighted')
    
    def classificationReport(self, X, y) :
        prediction = self.predict(X)
        return classification_report(y, prediction, output_dict=True)
    

In [3]:
dfTrain = pd.read_pickle('../data/dfTrain.pickle')
dfTest = pd.read_pickle('../data/dfTest.pickle')
print(dfTrain.info())
print()
print(dfTest.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3089 entries, 2270 to 372
Data columns (total 5 columns):
featureOne      3089 non-null float64
featureTwo      3089 non-null float64
featureThree    3089 non-null float64
featureFour     3089 non-null float64
label           3089 non-null int16
dtypes: float64(4), int16(1)
memory usage: 126.7 KB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 933 to 3687
Data columns (total 5 columns):
featureOne      4000 non-null float64
featureTwo      4000 non-null float64
featureThree    4000 non-null float64
featureFour     4000 non-null float64
label           4000 non-null int16
dtypes: float64(4), int16(1)
memory usage: 164.1 KB
None


In [4]:
trainData = dfTrain[['featureOne', 'featureTwo', 'featureThree', 'featureFour']].values
trainLabel = dfTrain['label'].values
print(trainData.shape)
print(trainLabel.shape)

(3089, 4)
(3089,)


In [5]:
testData = dfTest[['featureOne', 'featureTwo', 'featureThree', 'featureFour']].values
testLabel = dfTest['label'].values
print(testData.shape)
print(testLabel.shape)

(4000, 4)
(4000,)


In [6]:
# Perform GridSearch to search for the optimal value of the parameter C
# Step 1: Wide range Grid Search
params = {'C':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}


svm = SimpleSVM()
grid = GridSearchCV(estimator=svm, param_grid=params, cv=5)
grid.fit(trainData, trainLabel)
print(grid.best_params_)

Instructions for updating:
Colocations handled automatically by placer.


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:143

{'C': 1.0}


In [7]:
# Train SVM model with the Optimal Parameter value
model = SimpleSVM(C=1.0, verbose=True)
model.fit(trainData, trainLabel)

Epoch 0, Step 1    Loss: 365.20123291015625    F1_Score: 0.7272859859446053    Accuracy: 0.724
Epoch 0, Step 2    Loss: 211.04568481445312    F1_Score: 0.8444606732273002    Accuracy: 0.842
Epoch 0, Step 3    Loss: 152.81729125976562    F1_Score: 0.892322382262502    Accuracy: 0.892
Epoch 0, Step 4    Loss: 118.96159362792969    F1_Score: 0.9019369239593504    Accuracy: 0.902
Epoch 0, Step 5    Loss: 94.5107192993164    F1_Score: 0.9223213435299157    Accuracy: 0.922
Epoch 0, Step 6    Loss: 90.16957092285156    F1_Score: 0.9362956680629536    Accuracy: 0.936
Epoch 1, Step 1    Loss: 91.43257141113281    F1_Score: 0.930315380203248    Accuracy: 0.93
Epoch 1, Step 2    Loss: 85.33943176269531    F1_Score: 0.9342508818964302    Accuracy: 0.934
Epoch 1, Step 3    Loss: 98.49528503417969    F1_Score: 0.924654422442808    Accuracy: 0.924
Epoch 1, Step 4    Loss: 86.79888153076172    F1_Score: 0.9461009187500817    Accuracy: 0.946
Epoch 1, Step 5    Loss: 69.46237182617188    F1_Score: 0.960

Epoch 21, Step 6    Loss: 78.3308334350586    F1_Score: 0.948420403027012    Accuracy: 0.948
Epoch 22, Step 1    Loss: 75.97100830078125    F1_Score: 0.9640904459487752    Accuracy: 0.964
Epoch 22, Step 2    Loss: 81.60762786865234    F1_Score: 0.9438706452224578    Accuracy: 0.944
Epoch 22, Step 3    Loss: 79.71827697753906    F1_Score: 0.9558820957619351    Accuracy: 0.956
Epoch 22, Step 4    Loss: 95.99373626708984    F1_Score: 0.9327187670485544    Accuracy: 0.932
Epoch 22, Step 5    Loss: 63.954044342041016    F1_Score: 0.9596254355400697    Accuracy: 0.96
Epoch 22, Step 6    Loss: 66.49209594726562    F1_Score: 0.956262907220423    Accuracy: 0.956
Epoch 23, Step 1    Loss: 71.56950378417969    F1_Score: 0.9602647058823529    Accuracy: 0.96
Epoch 23, Step 2    Loss: 69.80047607421875    F1_Score: 0.9596524785927527    Accuracy: 0.96
Epoch 23, Step 3    Loss: 80.54708099365234    F1_Score: 0.9439999999999998    Accuracy: 0.944
Epoch 23, Step 4    Loss: 61.97605895996094    F1_Score

Epoch 42, Step 5    Loss: 64.06410217285156    F1_Score: 0.9580784923611748    Accuracy: 0.958
Epoch 42, Step 6    Loss: 76.08366394042969    F1_Score: 0.9578797732350167    Accuracy: 0.958
Epoch 43, Step 1    Loss: 78.72352600097656    F1_Score: 0.9599076037303271    Accuracy: 0.96
Epoch 43, Step 2    Loss: 73.05026245117188    F1_Score: 0.9522141570141571    Accuracy: 0.952
Epoch 43, Step 3    Loss: 64.50236511230469    F1_Score: 0.9640774973945431    Accuracy: 0.964
Epoch 43, Step 4    Loss: 82.7366943359375    F1_Score: 0.9436710687960688    Accuracy: 0.944
Epoch 43, Step 5    Loss: 65.64997100830078    F1_Score: 0.9581485855668637    Accuracy: 0.958
Epoch 43, Step 6    Loss: 70.07272338867188    F1_Score: 0.9599043973743917    Accuracy: 0.96
Epoch 44, Step 1    Loss: 73.95331573486328    F1_Score: 0.9560508179483201    Accuracy: 0.956
Epoch 44, Step 2    Loss: 66.84408569335938    F1_Score: 0.9540286559413812    Accuracy: 0.954
Epoch 44, Step 3    Loss: 89.38644409179688    F1_Sco

Epoch 65, Step 4    Loss: 72.08565521240234    F1_Score: 0.9443096993760635    Accuracy: 0.944
Epoch 65, Step 5    Loss: 74.220703125    F1_Score: 0.9601021496912769    Accuracy: 0.96
Epoch 65, Step 6    Loss: 71.3676528930664    F1_Score: 0.9599011469047496    Accuracy: 0.96
Epoch 66, Step 1    Loss: 74.02857208251953    F1_Score: 0.9405833333333334    Accuracy: 0.94
Epoch 66, Step 2    Loss: 85.2539291381836    F1_Score: 0.9440581159938073    Accuracy: 0.944
Epoch 66, Step 3    Loss: 68.84705352783203    F1_Score: 0.96391827187672    Accuracy: 0.964
Epoch 66, Step 4    Loss: 77.76630401611328    F1_Score: 0.9540249795827614    Accuracy: 0.954
Epoch 66, Step 5    Loss: 66.55121612548828    F1_Score: 0.9621911200960375    Accuracy: 0.962
Epoch 66, Step 6    Loss: 65.38790893554688    F1_Score: 0.952    Accuracy: 0.952
Epoch 67, Step 1    Loss: 72.25934600830078    F1_Score: 0.9599553839553839    Accuracy: 0.96
Epoch 67, Step 2    Loss: 77.64058685302734    F1_Score: 0.9539694276823402 

Epoch 88, Step 3    Loss: 103.25054168701172    F1_Score: 0.9242322580645161    Accuracy: 0.924
Epoch 88, Step 4    Loss: 51.13027572631836    F1_Score: 0.9700666591852767    Accuracy: 0.97
Epoch 88, Step 5    Loss: 67.1559066772461    F1_Score: 0.9679584736251403    Accuracy: 0.968
Epoch 88, Step 6    Loss: 58.368080139160156    F1_Score: 0.9640505519116941    Accuracy: 0.964
Epoch 89, Step 1    Loss: 78.7167739868164    F1_Score: 0.9599569458919205    Accuracy: 0.96
Epoch 89, Step 2    Loss: 60.39820098876953    F1_Score: 0.9582239578603214    Accuracy: 0.958
Epoch 89, Step 3    Loss: 78.17385864257812    F1_Score: 0.9480674803591469    Accuracy: 0.948
Epoch 89, Step 4    Loss: 66.40443420410156    F1_Score: 0.9542839694656489    Accuracy: 0.954
Epoch 89, Step 5    Loss: 68.05574798583984    F1_Score: 0.9698922255097532    Accuracy: 0.97
Epoch 89, Step 6    Loss: 93.2598876953125    F1_Score: 0.9381216598843718    Accuracy: 0.938
Epoch 90, Step 1    Loss: 60.791969299316406    F1_Sco

...

Epoch 433, Step 6    Loss: 77.21930694580078    F1_Score: 0.9582674540203543    Accuracy: 0.958
Epoch 434, Step 1    Loss: 71.26094818115234    F1_Score: 0.9579746921046131    Accuracy: 0.958
Epoch 434, Step 2    Loss: 83.77067565917969    F1_Score: 0.9461388092233567    Accuracy: 0.946
Epoch 434, Step 3    Loss: 77.6402359008789    F1_Score: 0.9543651594416626    Accuracy: 0.954
Epoch 434, Step 4    Loss: 74.51163482666016    F1_Score: 0.9517627525713988    Accuracy: 0.952
Epoch 434, Step 5    Loss: 77.64735412597656    F1_Score: 0.948458691910499    Accuracy: 0.948
Epoch 434, Step 6    Loss: 77.73390197753906    F1_Score: 0.9519040309053016    Accuracy: 0.952
Epoch 435, Step 1    Loss: 71.27801513671875    F1_Score: 0.950102887487554    Accuracy: 0.95
Epoch 435, Step 2    Loss: 73.45555114746094    F1_Score: 0.9540888213633639    Accuracy: 0.954
Epoch 435, Step 3    Loss: 74.8650894165039    F1_Score: 0.9461128639322657    Accuracy: 0.946
Epoch 435, Step 4    Loss: 63.25193405151367 

Epoch 450, Step 3    Loss: 74.87968444824219    F1_Score: 0.9539659297497476    Accuracy: 0.954
Epoch 450, Step 4    Loss: 82.15526580810547    F1_Score: 0.9460992616863448    Accuracy: 0.946
Epoch 450, Step 5    Loss: 73.50519561767578    F1_Score: 0.9540670256572696    Accuracy: 0.954
Epoch 450, Step 6    Loss: 65.89728546142578    F1_Score: 0.9700225562894229    Accuracy: 0.97
Epoch 451, Step 1    Loss: 81.805908203125    F1_Score: 0.945916714465986    Accuracy: 0.946
Epoch 451, Step 2    Loss: 64.67162322998047    F1_Score: 0.9641312499999999    Accuracy: 0.964
Epoch 451, Step 3    Loss: 75.27269744873047    F1_Score: 0.9501045036409866    Accuracy: 0.95
Epoch 451, Step 4    Loss: 71.50665283203125    F1_Score: 0.953967952216764    Accuracy: 0.954
Epoch 451, Step 5    Loss: 69.22135925292969    F1_Score: 0.9601140819964349    Accuracy: 0.96
Epoch 451, Step 6    Loss: 70.02376556396484    F1_Score: 0.9578046030132175    Accuracy: 0.958
Epoch 452, Step 1    Loss: 70.96078491210938   

Epoch 472, Step 4    Loss: 89.0225830078125    F1_Score: 0.9443575239194697    Accuracy: 0.944
Epoch 472, Step 5    Loss: 66.84080505371094    F1_Score: 0.9558983641033596    Accuracy: 0.956
Epoch 472, Step 6    Loss: 72.77072143554688    F1_Score: 0.9366081811727439    Accuracy: 0.936
Epoch 473, Step 1    Loss: 78.86087036132812    F1_Score: 0.9520612898147662    Accuracy: 0.952
Epoch 473, Step 2    Loss: 70.26863861083984    F1_Score: 0.9679156043956045    Accuracy: 0.968
Epoch 473, Step 3    Loss: 91.91069793701172    F1_Score: 0.9285788026232283    Accuracy: 0.928
Epoch 473, Step 4    Loss: 77.34317779541016    F1_Score: 0.9516417974861089    Accuracy: 0.952
Epoch 473, Step 5    Loss: 65.981201171875    F1_Score: 0.9583015384615385    Accuracy: 0.958
Epoch 473, Step 6    Loss: 64.7657470703125    F1_Score: 0.9699096457260298    Accuracy: 0.97
Epoch 474, Step 1    Loss: 67.35585021972656    F1_Score: 0.9622434688814583    Accuracy: 0.962
Epoch 474, Step 2    Loss: 62.80795669555664 

Epoch 494, Step 1    Loss: 76.06788635253906    F1_Score: 0.9523617837951008    Accuracy: 0.952
Epoch 494, Step 2    Loss: 69.5902328491211    F1_Score: 0.96    Accuracy: 0.96
Epoch 494, Step 3    Loss: 71.20645904541016    F1_Score: 0.9523085689239535    Accuracy: 0.952
Epoch 494, Step 4    Loss: 74.31056213378906    F1_Score: 0.9462782390846753    Accuracy: 0.946
Epoch 494, Step 5    Loss: 75.37025451660156    F1_Score: 0.9580297171133727    Accuracy: 0.958
Epoch 494, Step 6    Loss: 74.55577087402344    F1_Score: 0.9558395833333334    Accuracy: 0.956
Epoch 495, Step 1    Loss: 73.69559478759766    F1_Score: 0.9561825014180375    Accuracy: 0.956
Epoch 495, Step 2    Loss: 71.17811584472656    F1_Score: 0.9639004537719796    Accuracy: 0.964
Epoch 495, Step 3    Loss: 80.30267333984375    F1_Score: 0.9404358699535549    Accuracy: 0.94
Epoch 495, Step 4    Loss: 74.03557586669922    F1_Score: 0.9560448217317487    Accuracy: 0.956
Epoch 495, Step 5    Loss: 73.24553680419922    F1_Score:

SimpleSVM(C=1.0, verbose=True)

In [8]:
modelReportOnTest = model.classificationReport(testData, testLabel)
print(modelReportOnTest)
with open('../data/simpleSvmReport.pickle', 'wb') as fp :
    pickle.dump(modelReportOnTest, fp)
print('Report Saved')

{'-1': {'precision': 0.9623792577529232, 'recall': 0.9465, 'f1-score': 0.9543735820519283, 'support': 2000}, '1': {'precision': 0.9473684210526315, 'recall': 0.963, 'f1-score': 0.9551202578725514, 'support': 2000}, 'accuracy': 0.95475, 'macro avg': {'precision': 0.9548738394027774, 'recall': 0.95475, 'f1-score': 0.9547469199622398, 'support': 4000}, 'weighted avg': {'precision': 0.9548738394027774, 'recall': 0.95475, 'f1-score': 0.9547469199622399, 'support': 4000}}
Report Saved
